<a href="https://colab.research.google.com/github/luiscaballeroramos/DeCoder/blob/EC3_1_8_DesignOfJoints/DeCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminary Tasks

Install and import PyPDF2 library to open and read the PDF file containing the code

In [405]:
!pip install PyPDF2
import PyPDF2

Clone the GitHub repository containing the PDF file with code

In [406]:
!git clone -b EC3_1_8_DesignOfJoints https://github.com/luiscaballeroramos/DeCoder.git

fatal: destination path 'DeCoder' already exists and is not an empty directory.


Import libraries used in this tool

In [408]:
import numpy as np
import regex as re
from tabulate import tabulate

# 1. Read PDF

Read the .pdf file with PyPF2 library

In [409]:
pdfFile=open('DeCoder/EC 1993.1.8.2005-1.pdf','rb')
pdfReader=PyPDF2.PdfFileReader(pdfFile)

In [410]:
numPages=pdfReader.numPages
print(numPages)

135


Get the pages content as dictionary (pages) with:
* key = page number (pageNum)
* value = page content as string (pageContent)



In [411]:
pages={}
pageContent=''
for pageNum in range(0,numPages):
  pdfPage=pdfReader.getPage(pageNum)
  pageContent=pdfPage.extractText()
  pages[pageNum]=pageContent
  pass

# 2. Table Of Contents (TOC)
2.0. TOC Definition (settings)

2.1. TOC Location (pages)

2.2. TOC Contents (toc's)

# 2.0. TOC Definition

Set TOC identifiers, keywords that could be used to refering TOC

In [412]:
TOCidentifiers=['contents','Contents',
                 'table of contents','Table of contents','Table Of Contents']

Description of toc elements by regex expressions 

In [437]:
refidRegex='\d'
refsepRegex='\.'
namRegex='[\s\w]*'
sepRegex='\.{2,}'
numRegex='\d+'

In [474]:
class TOC():
  def __init__(self,refidRegex,refsepRegex,namRegex,sepRegex,numRegex):
    self.refidRegex=refidRegex
    self.refsepRegex=refsepRegex
    self.refRegex=refidRegex+'(?:'+refsepRegex+'+'+refidRegex+'*)+'
    self.namRegex=namRegex
    self.sepRegex=sepRegex
    self.numRegex=numRegex
    self.tocRegex=refRegex+'\s*'+namRegex+'\s*'+sepRegex+'\s*'+numRegex
    self.tocList=[]
    pass
  def addToc(self,toc):
    if len(list(x for x in self.tocList if x.ref == toc.ref)) == 0:
      self.tocList.append(toc)
      pass
    pass
  def printTOC(self,num=None):
    table=[]
    if num is None:
      for toc in TOC_EC3.tocList:
        table.append([toc.ref,toc.name,toc.begin.page,toc.level])
        pass
    else:
      toc=TOC_EC3.tocList[num]
      table.append([toc.ref,toc.name,toc.begin.page,toc.level])
      pass
    print(tabulate(table,headers=['Ref.','Name','Begin page','Level'], tablefmt='orgtbl'))
    pass
  pass
TOC_EC3=TOC(refidRegex,refsepRegex,namRegex,sepRegex,numRegex)

In [475]:
class PDFlocation():
  def __init__(self,page,begin=None,end=None):
    self.page=page
    if begin is None:
      self.begin=None
      self.end =None
    else:
      self.begin=int(begin)
      if end is None:
        self.end=int(begin)
      else:
        self.end=end
        pass
      pass
    pass
  pass

In [476]:
class toc():
  def __init__(self,ref,name,begin=None,end=None,level=None):
    self.ref=ref
    self.name=name
    self.begin=begin
    self.end=end
    self.setLevel()
    pass
  def setLevel(self):
    self.level = len(list(filter(lambda item: item, re.split('\.*',self.ref))))-1
    pass
  pass

# 2.1. TOC Location

TOC identifiers apearances in each document page 

In [417]:
TOCcoincidences={}
TOCtotalCoincidences={}
for pageNum, pageContent in pages.items():
  aux=[]
  for TOCidentifier in TOCidentifiers:
    aux.append(pageContent.count(TOCidentifier))
    pass
  TOCcoincidences[pageNum]=aux
  TOCtotalCoincidences[pageNum]=sum(aux)
  pass

Total of TOC identifiers apearances in PDF document

In [418]:
sumTOCTotalCoincidences=sum(TOC_totalCoincidences.values())
print('Number of TOC indentifiers: {0}'.format(sumTOCTotalCoincidences))

Number of TOC indentifiers: 1


Max TOC identifiers apearances in one page and the page where this happens

In [419]:
maxTOCTotalCoincidences=max(TOCtotalCoincidences.values())
maxTOCTotalCoincidencesPages = [key  for (key, value) in TOC_totalCoincidences.items() if value == maxTotalCoincidences]
print('Number of Max TOC indentifiers in One Page: {0}'.format(maxTotalCoincidences))
print('Pages with Max TOC indentifiers in One Page: {0}'.format(maxTotalCoincidencesPages))

Number of Max TOC indentifiers in One Page: 1
Pages with Max TOC indentifiers in One Page: [3]


Pages where toc pattern (Regex) is detected

In [420]:
tocCoincidences={}
for pageNum, pageContent in pages.items():
  tocCoincidences[pageNum]=len(re.findall(TOC_EC3.tocRegex,pages[pageNum]))
  pass

In [421]:
noNulltocCoincidencesPages = [key  for (key, value) in tocCoincidences.items() if value != 0]
print('Pages with toc Regex: {0}'.format(noNulltocCoincidencesPages))

Pages with toc Regex: [3, 4, 5]


In [422]:
TOCpages=noNulltocCoincidencesPages

# 2.2. TOC Contents

Look for complete tocRegex in TOCpages

In [477]:
pattern = re.compile(TOC_EC3.tocRegex)
for page in TOCpages:
  for match in pattern.finditer(pages[page]):
    [split1,split2]=re.split(TOC_EC3.sepRegex,match.group())
    ref=re.search(TOC_EC3.refRegex,split1).group()
    split1=re.sub(TOC_EC3.refRegex,'',split1)
    nam=re.search(TOC_EC3.namRegex,split1).group()
    split1=re.sub(TOC_EC3.namRegex,'',split1)
    num=re.search(TOC_EC3.numRegex,split2).group()
    beginLocation=PDFlocation(num)
    toc_EC3=toc(ref,nam,beginLocation)
    TOC_EC3.addToc(toc_EC3)
    pass
  pass
TOC_EC3.tocList.sort(key=lambda x: x.ref, reverse=False)
TOC_EC3.printTOC()

| Ref.   | Name                                                                           |   Begin page |   Level |
|--------+--------------------------------------------------------------------------------+--------------+---------|
| 1.3    | Distinction between Principles and Application Rules                           |            1 |       1 |
| 1.5    | SY111bols                                                                      |           13 |       1 |
| 2.1    | AsslIlnptions                                                                  |           18 |       1 |
| 2.2    | General requirenlents                                                          |           18 |       1 |
| 2.4    | Resistance of joints                                                           |           18 |       1 |
| 2.5    | Design assunlptions                                                            |           19 |       1 |
| 2.7    | Eccentricity at intersections                        

Commplete with references that can be deduced to exist in TOC

In [480]:
TOC_EC3.printTOC(0)
toc=TOC_EC3.tocList[0]

|   Ref. | Name                                                 |   Begin page |   Level |
|--------+------------------------------------------------------+--------------+---------|
|    1.3 | Distinction between Principles and Application Rules |            1 |       1 |


# Close PDF

In [ ]:
pdfFile.close()